In [35]:
import os
import pymysql
import pandas as pd
from dotenv import load_dotenv, find_dotenv



In [36]:
env_path = find_dotenv()
load_dotenv(env_path)
#print(os.getenv('HOST'))
#print(os.getenv('USER_DB'))

True

In [37]:
# access to the data
conn = pymysql.connect(
    host=os.getenv('HOST'),
    port=int(3306),
    user=os.getenv('USER_DB'),
    passwd=os.getenv('PASSWORD'),
    db=os.getenv('DB'),
    charset='utf8mb4')



## Easy features

-Resources added to queue

-Resources completed from queue

-Ratio of resources added to queue to completed

-Minutes learned per year



The first three is in the queue.py file. The last is in the time_learning.py

In [9]:
df_trk = pd.read_sql_query("SELECT * FROM tracking_event;", conn).drop_duplicates()
df_res = pd.read_sql_query("SELECT * FROM resource;", conn).drop_duplicates()

df_trk_copy=df_trk.copy()
df_res_copy=df_res.copy()

/Users/yayahuo/.pyenv/versions/3.8.13/envs/customerclustering/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/yayahuo/.pyenv/versions/3.8.13/envs/customerclustering/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [39]:
import numpy as np
# clean track_event

# reorder the dataframe according to the userID and eventDate
df_trk=df_trk_copy
df_trk.isna().sum()/len(df_trk) #content & skipMigration have over 80% missing values, delete these columns. And drop eventID
df_trk=df_trk.iloc[1:,:-2]
#df_trk['eventType'].apply(lambda)


# reorder 
df_trk=df_trk.sort_values(by=['userID','eventDate'],ascending=[True,True]).reset_index(drop=True)
df_trk
#-------------------------------------------------------------------------------------------

# return a dataframe with userID, eventType, source,eventDate, resourceType and action
# Get queueing status (returns 1 if completed from queue)
df_trk['resourceType']=df_trk['eventType'].apply(lambda x: x.split('_')[0])
df_trk['action']=df_trk['eventType'].apply(lambda x: x.split('_')[1])
df_trk.head(20)

# Get CompletionFromQueue (returns 1 if completed from the queue)


# Get mins learned per year(make this in a different py)



#Get resourceID and source
df_src=df_trk_copy[['resourceID','source']].drop_duplicates()
# create a dataframe with userID, numOfResourcesToQueue, numOfCompletionFromQueue, RatioOfCompletion_num, minOfResourcesToQueue, minOfCompletedFromQueue, RatioOfCompletion_min
# create "isQueued" columns, returns 1 if Queued
df_trk['isQueued']=df_trk['action'].map({'queued':1}).fillna(0).astype(int)
# create "isCompleted" columns, returns 1 if Completed
df_trk['isCompleted']=df_trk['action'].map({'completed':1, 'documented':1}).fillna(0).astype(int)

df_trk=df_trk.groupby(['userID','resourceID']).sum().reset_index()
# Get CompletionFromQueue (returns 1 if completed from the queue)
df_trk['CompletedFromQueue']=((df_trk['isQueued']==1)&(df_trk['isCompleted']==1)).astype(int)


df_trk=df_trk.merge(df_src,on='resourceID')


df_trk.head(20)

# add mins from the resourceID
df_res_select=df_res[['resourceID','min']].drop_duplicates()
df_trk=df_trk.merge(df_res_select,on='resourceID')
df_trk['minQueued']=df_trk['isQueued']*df_trk['min']
df_trk['minCompleted']=df_trk['CompletedFromQueue']*df_trk['min']


# if 'specified_source=True'
df_trk1=df_trk.groupby(['userID','source']).sum().reset_index()
df_trk1.rename(columns={'isQueued':'numQueued','CompletedFromQueue':'numCompletedFromQueue'},inplace=True)


df_trk1.head()

# if 'specified_source=False'
df_trk2=df_trk1.groupby(['userID']).sum().reset_index()
df_trk2['numQueued']=df_trk2['numQueued'].replace(0,-1)
df_trk2['minQueued']=df_trk2['minQueued'].replace(0,-1)
df_trk2['RatioOfCompletion_num']=np.abs(df_trk2['numCompletedFromQueue']/df_trk2['numQueued'])
df_trk2['RatioOfCompletion_min']=np.abs(df_trk2['minCompleted']/df_trk2['minQueued'])
# drop 'min' and 'isCompleted' as unnecessary
df_trk2=df_trk2.drop(['min','isCompleted'],axis=1)
#replace -1 back to 0
df_trk2['numQueued']=df_trk2['numQueued'].replace(-1,0)
df_trk2['minQueued']=df_trk2['minQueued'].replace(-1,0)
df_trk2.head(10)
#------------------------------------------------------------------

# if 'specified_source=True'
df_trk1['numQueued']=df_trk1['numQueued'].replace(0,-1)
df_trk1['minQueued']=df_trk1['minQueued'].replace(0,-1)
df_trk1['RatioOfCompletion_num']=np.abs(df_trk1['numCompletedFromQueue']/df_trk1['numQueued'])
df_trk1['RatioOfCompletion_min']=np.abs(df_trk1['minCompleted']/df_trk1['minQueued'])
#replace -1 back to 0
df_trk1['numQueued']=df_trk1['numQueued'].replace(-1,0)
df_trk1['minQueued']=df_trk1['minQueued'].replace(-1,0)
# drop min and 'isCompleted'
df_trk1=df_trk1.drop(['min','isCompleted'],axis=1)




In [36]:
df_trk1.head(20)

,userID,source,numQueued,numCompletedFromQueue,minQueued,minCompleted,RatioOfCompletion_num,RatioOfCompletion_min
0,00001d9c-5de5-4ed2-9ecc-98341286765a,ausmed,12,0,700.0,0.0,0.000000,0.000000
1,0000c854-c7a3-4047-9a54-2e1a3464621f,ausmed,-1,0,-1.0,0.0,0.000000,0.000000
2,000169ca-f6bf-4ac0-a13d-c2b188014962,ausmed,-1,0,-1.0,0.0,0.000000,0.000000
3,0001c897-b1fe-40e0-afef-6a667edf41f7,ausmed,-1,0,-1.0,0.0,0.000000,0.000000
4,0002e8b5-261b-44a6-b38c-2997b4007317,ausmed,-1,0,-1.0,0.0,0.000000,0.000000
5,0002f0ba-44b0-44d3-a542-3be3dd4d5036,ausmed,1,0,50.0,0.0,0.000000,0.000000
6,00034f05-a97f-4dfe-8162-f83b7709b392,ausmed,-1,0,-1.0,0.0,0.000000,0.000000
7,00036835-5926-476a-bd8a-65c39be4e6cc,ausmed,69,1,2009.0,7.0,0.014493,0.003484
8,0003f9de-385c-45dc-9b19-31df24266aa9,ausmed,-1,0,-1.0,0.0,0.000000,0.000000
9,00049633-8b5c-4a00-b869-4c107f941d13,ausmed,-1,0,-1.0,0.0,0.000000,0.000000


##Easy Feature Min per year

Two potential resources:

-'activity_20220808' (use this)

-tracking_event

-

Two Steps:
1.create a column to get user's Ausmed year (from user AusmedYear=lastest activity date-account createDate)
2.caculate the total min

In [38]:
# check activity data
nrow=200000
df_act=pd.read_sql_query(f"SELECT * FROM activity_20220808 LIMIT {nrow};", conn).drop_duplicates()
df_usr=pd.read_sql_query("SELECT * FROM user;", conn).drop_duplicates()

df_act_copy=df_act.copy()
df_usr_copy=df_usr.copy()

/Users/yayahuo/.pyenv/versions/3.8.13/envs/customerclustering/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/yayahuo/.pyenv/versions/3.8.13/envs/customerclustering/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [55]:
import datetime as dt

# clean data here
# create a data frame with 'userID', 'activityType', 'providerName', 
df_act=df_act_copy
df_act['providerName'].unique()
#df_act['status'].unique()

# create minPerYear
## step1: get total min
selected_columns=['owner','providerName','min','updateDate','completeDate']
df_act=df_act[selected_columns]
#replace missing values of  'updateDate'
df_act['completeDate']=df_act['completeDate'].fillna(df_act['updateDate'])

# fill in missing value with 'Unknown'
# replace 'N/A' by 'Unknown'
df_act['providerName']=df_act['providerName'].fillna('Unknown')
df_act['providerName'].replace('N/A','Unknown',inplace=True)



# rename 'owner' as 'userID'
df_act.rename(columns={'owner':'userID'},inplace=True)
df_act.head(20)
##################################################################################

# get yearsOnAusmed 
df_usr=df_usr_copy[['userID','createDate']]
df_usr=df_usr.merge(df_act, on='userID', how='inner')
df_usr['yearsOnAusmed']=df_usr['completeDate']-df_usr['createDate']
df_usr.head()
#####################################################################################
# get features here
# returns a data frame with userID,favoriteActivityType, 2ndfavoriteActivityType,minPerYear,percentageLearningFromAusmed
# create two columns to get favoriteActivityType, 2ndfavoriteActivityType
#define a function to get the nth/2nd most frequent activityType
def fav_activityType(series,n=2):
    ls=series.value_counts().index.tolist()[n-1]
    return ls



# df_act.groupby('owner').agg({})   

# create a column "isAusmed": returns 1 if the provider is Ausmend
#df_act['isAusmed']=df_act['providerName'].apply(lambda x: 1 if ('Ausmed' in x) else 0 )

# create a copy of acticityType to get 2nd favorite activityType


# # create two columns to get percentageLearningFromAusmed_Min, percentageLearningFromAusmed_num
   
# # rename the columns


# # get min
# df_act['providerName'].unique()

# #df_act.columns


/var/folders/bh/j223y_td6s35mkxwj_m8bxdm0000gn/T/ipykernel_24869/3272922448.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_act['completeDate']=df_act['completeDate'].fillna(df_act['updateDate'])
/var/folders/bh/j223y_td6s35mkxwj_m8bxdm0000gn/T/ipykernel_24869/3272922448.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_act['providerName']=df_act['providerName'].fillna('Unknown')
/var/folders/bh/j223y_td6s35mkxwj_m8bxdm0000gn/T/ipykernel_24869/3272922448.py:19: SettingWithCopyWarning: 
A value

In [57]:
import datetime as dt
#df_act[df_act['providerName']=='N/A']
#df_usr.head()
#df_usr['yearsOnAusmed'].dt.to_period('Y')
df_act_copy.head(20)

,activityID,owner,title,activityType,goals,qualifications,resource,providerName,startDate,endDate,...,authorNames,publisher,purpose,involvement,outcome,focus,colleague,topicTags,competencyStandards,accreditations
0,00000397-b19c-4578-8413-d87fe9d2d22f,71cc3c35-1a98-4992-bafe-ee711cfc7b91,Bullying and Harrassment of Employees,Competency,None,0b704058-194d-4420-b2f2-4f06b0c25761,None,None,2020-09-18 04:00:00,2020-09-19 03:59:59,...,None,None,None,None,None,None,None,Bullying|Harrassement in the workplace,0a982b98-a3a1-4510-b914-44e78ff7a9d1,None
1,00000576-25d0-49ea-a7ec-24210d3f1f27,eaab229d-560b-4f3a-93e7-53b9709ff2e6,High-Risk Medicines: Avoiding Common Interactions,e-Learning,None,b03680d4-c726-4a19-85c4-7f2fb9f20674,8e604196-e65f-4e98-a699-c8807b0e00ef,Ausmed,2021-04-22 14:00:00,2021-04-23 13:59:59,...,Louis Roller,None,None,None,None,None,None,Medications|Patient Safety|Documentation,"8b88a895-c0ac-457b-aeeb-47aff4468e98,f591f9f6-...",None
2,00000599-3407-4390-ac74-7aefa7c9c234,53517576-bb8f-4ce8-b886-f74682d067db,Paediatrics: Assessing the Febrile Infant or C...,e-Learning,None,0b704058-194d-4420-b2f2-4f06b0c25761,None,Ausmed Education,2017-04-15 04:00:00,2017-04-15 04:00:00,...,None,None,None,None,None,None,None,None,None,None
3,0000117f-9261-4874-a63c-d47e1e8fb824,f18f24ca-2a6b-4730-a315-d0590e86d048,Introduction to alcohol and other drugs,e-Learning,None,b03680d4-c726-4a19-85c4-7f2fb9f20674,None,ANMF,2022-02-16 13:00:00,2022-02-17 12:59:59,...,None,None,None,None,None,None,None,Alcohol|Drugs|Addiction and Substance Abuse,None,None
4,00001407-9ade-4f7c-890a-bb67e802b5b0,2cbd6711-946b-4384-824f-12f4f5023fed,Manual Handling Train the Trainer,In-Service,None,b03680d4-c726-4a19-85c4-7f2fb9f20674,None,None,2018-02-18 02:00:00,2018-02-19 01:59:59,...,None,None,None,None,None,None,None,None,None,None
5,0000140d-3d05-44a8-8676-e19540aa2da3,2c7e3a77-8e50-4bc7-973c-b30baf86a116,ABCDE Assessment,e-Learning,None,0b704058-194d-4420-b2f2-4f06b0c25761,None,Ausmed,2021-10-31 14:00:00,2021-11-01 13:59:59,...,None,None,None,None,None,None,None,ABCDE assessment,5d9a324b-34ff-4a75-8f08-89e37337b7b4,None
6,00001c4a-9e27-4ec8-be75-551323431b2f,8fcd4857-e620-4e4e-acf9-972a8fa07773,Duty of Care,e-Learning,None,None,da20b048-c730-4d71-bf34-638ce84b0b5d,None,2021-12-27 13:00:00,2021-12-28 12:59:59,...,None,None,None,None,None,None,None,Aged Care|Ageing|Allied Health|Community Nursi...,None,None
7,0000255b-56f9-4d34-9a3b-80ad45d70387,e62dab74-2092-45b0-8142-9eca730013c8,Iron Deficiency,Textbook,None,49c54b8a-499d-4dc7-8a03-b1bab580db54,f531472b-2cf5-4d2d-a3fd-a868ed763541,Ausmed,2022-03-25 13:00:00,2022-03-26 12:59:59,...,Ausmed,None,None,None,None,None,None,Anatomy and Physiology|Blood|Diet|Nutrition|Pr...,None,None
8,00002a2d-31d1-43ff-9c73-740ac4751140,79321abf-2019-424a-9ec7-1cda642bbe7c,Catheterise with confidence,Conference,None,0b704058-194d-4420-b2f2-4f06b0c25761,None,,2017-07-18 20:44:45,2017-07-18 20:44:45,...,None,None,None,None,None,None,None,Catheter Laboratory,None,None
9,00003029-61ce-476a-84c8-26c07cc73f1b,f0911702-ecbf-48d2-be41-e22ac4b9d024,Mentoring on 16/10/2018,Mentoring,None,b03680d4-c726-4a19-85c4-7f2fb9f20674,None,Utas,2018-10-15 02:00:00,2018-10-16 01:59:59,...,None,None,None,None,None,None,None,Medication administration|General nursing care...,None,None


# Hidden features

## Task1: % of users compliant per year (consdier hours of CPD completed and documented, learning goals set, context of practice set)

-CPD requirement is in the "groupRegulation"

-Goal: replacing intented 

-Q: how do we define overachiver/ casual learner/ ambitious learner

In [ ]:
# clean context of practice set

#create a table return CPD record with CPD hours required, CPD completed, documented

#create a goal table with column, goals per year, number of goals achived, ratio of goals achieved, ratio of 

# make compliency() function returns 

In [ ]:
# clean context of practice set
#load the data
df_cop = pd.read_sql_query("SELECT * FROM contextOfPractice;", conn)
df_pp=pd.read_sql_query("SELECT * FROM professionalprofile;", conn).drop_duplicates()
df_wk=pd.read_sql_query("SELECT * FROM work;", conn).drop_duplicates()
df_edu=pd.read_sql_query("SELECT * FROM education;", conn).drop_duplicates()
df_usr=pd.read_sql_query("SELECT * FROM user;", conn).drop_duplicates()



df_cop_copy=df_cop.copy()
df_pp_copy=df_pp.copy()
df_wk_copy=df_wk.copy()
df_edu_copy=df_edu.copy()
df_usr_copy=df_usr.copy()

/Users/yayahuo/.pyenv/versions/3.8.13/envs/customerclustering/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/yayahuo/.pyenv/versions/3.8.13/envs/customerclustering/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/yayahuo/.pyenv/versions/3.8.13/envs/customerclustering/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/yayahuo/.pyenv/versio

In [ ]:
# sneak peek 
import datetime as dt
df_cop=df_cop_copy
# df_cop=df_cop[['pProfile', 'typeOfPractice', 'located', 'specialities',
#        'population', 'focus', 'complex', 'autonomy', 'access', 'startDate',
#        'endDate', 'createDate', 'updateDate', 'topics', 'summary']]
#print(df_cop.columns)
#check
df_cop.isna().sum()/len(df_cop)# drop['topics', 'summary']
df_cop=df_cop[['pProfile', 'typeOfPractice', 'located', 'specialities',
       'population', 'focus', 'complex', 'autonomy', 'access', 'startDate',
       'endDate','createDate']]
# convert 'startDate','endDate'
#df_cop['startDate']=pd.to_datetime(df_cop['startDate'],errors='coerce')  

#df_cop[df_cop['startDate'].isna()]
df_cop.iloc[[3575,16699,29224,51510]]

# df_cop.iloc[[3575]]['startDate'] is '-2019-12-09' must have been typo, let's change that!
df_cop.at[3575,'startDate']='2019-12-09'
#df_cop.iloc[[3575]]
df_cop['startDate']=pd.to_datetime(df_cop['startDate'],errors='coerce')  
df_cop['createDate']=pd.to_datetime(df_cop['createDate'],errors='raise')  
df_cop[df_cop['startDate'].isna()]

# compute gap between createDate and startDate
#df_cop['gapBetweenStartAndCreate']=df_cop['createDate']-df_cop['startDate']
df_cop.head()
median=(df_cop['createDate']-df_cop['startDate']).median()
df_cop['startDate'].fillna(value=df_cop['createDate']-median,inplace=True)
df_cop['startDate']=df_cop['startDate'].apply(lambda x: dt.datetime(x.year,x.month,x.day))
#check previsously Nat rows
df_cop.iloc[[3575,16699,29224,51510]]

# get years of experience 
## replace missing values in endDate by current date
df_cop['endDate'].fillna(value=dt.date.today(),inplace=True)
df_cop['endDate']=pd.to_datetime(df_cop['endDate'],errors='coerce')
df_cop['yearsOfExperience']=df_cop['endDate']-df_cop['startDate']
#df_cop.info()

# link with userID
df_pp_select=df_pp[['pProfileID', 'owner','country']].drop_duplicates()
df_pp_select.head()
merged_df=df_cop.merge(df_pp_select,left_on='pProfile',right_on='pProfileID')\
.drop_duplicates().rename(columns={'owner':'userID'})# rename 'owner' as 'userID'

# drop 'pProfile' as redundant, move usersID and pProfileID to the front
merged_df=merged_df.drop(['pProfile','endDate','createDate'],axis=1)
cols = list(merged_df)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('pProfileID')))
cols.insert(0, cols.pop(cols.index('userID')))

merged_df = merged_df.loc[:, cols]
list(merged_df)

['userID',
 'pProfileID',
 'typeOfPractice',
 'located',
 'specialities',
 'population',
 'focus',
 'complex',
 'autonomy',
 'access',
 'country']

In [45]:
# main
# This table has one-to-many relationships, keep the lastest 
# return userID,typeOfPractice, located	specialities, population, focus, complex autonomy access startDate, yearsOfExperience
df_cop = pd.read_sql_query("SELECT * FROM contextOfPractice;", conn).drop_duplicates()
df_pp=df_pp_copy



# drop duplciates
df_cop = df_cop.drop_duplicates()

df_cop=df_cop[['pProfile', 'typeOfPractice', 'located', 'specialities',
       'population', 'focus', 'complex', 'autonomy', 'access','startDate','createDate','endDate']]

# df_cop.iloc[[3575]]['startDate'] is '-2019-12-09' must have been typo, let's change that!
df_cop.at[3575,'startDate']='2019-12-09'

# link with userID
df_pp_select=df_pp[['pProfileID', 'owner','country']].drop_duplicates()
df_pp_select.head()
merged_df=df_cop.merge(df_pp_select,left_on='pProfile',right_on='pProfileID')\
.drop_duplicates().rename(columns={'owner':'userID'})# rename 'owner' as 'userID'

# drop 'pProfile' as redundant, move usersID and pProfileID to the front
merged_df=merged_df.drop(['pProfile'],axis=1)
cols = list(merged_df)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('pProfileID')))
cols.insert(0, cols.pop(cols.index('userID')))


merged_df = merged_df.loc[:, cols].drop_duplicates().sort_values(by=['userID','startDate'],ascending=[True,False])

# only keep the lastest
merged_df.drop_duplicates(subset=['userID'],keep='first',ignore_index=True,inplace=True)
#merged_df.head(10)

# drop "startDate, createDate, endDate" as they are no longer needed
merged_df.drop(['startDate', 'createDate', 'endDate'],axis=1)
merged_df.head(10)


/Users/yayahuo/.pyenv/versions/3.8.13/envs/customerclustering/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,userID,pProfileID,typeOfPractice,located,specialities,population,focus,complex,autonomy,access,startDate,createDate,endDate,country
0,0000c854-c7a3-4047-9a54-2e1a3464621f,694b3b93-e762-427b-a1b8-72426d764a6c,private company,metropolitan centre,Perioperative Nursing,adults,clinical practice,generally complex,moderate,usually,2020-07-15,2020-07-14 21:46:21,None,AUS
1,00013cfc-f97f-428d-bf6b-558601512bd7,0bbfbfc8-5b48-4ca1-b05f-21638761d716,Other,metropolitan centre,"Immunisation,Cancer screening,General Practice",Other,nurse education,generally complex,moderate,usually,2017-06-01,2017-08-09 01:08:25,2018-05-31,AUS
2,00023498-c236-45f6-aaf1-ee382e520737,abcef1bf-aff5-45a8-b4e7-8c6a4a74a3d1,community,large rural centre,Paramedicine,adults,clinical practice,high complexity,moderate,usually,2018-12-01,2019-11-07 20:12:17,2019-11-30,AUS
3,0002f0ba-44b0-44d3-a542-3be3dd4d5036,5e540d39-66bc-422f-bd37-8b98ae0efd17,hospital,large rural centre,Surgical,adolescents or youth,clinical practice,generally complex,minimal,always,2022-02-01,2020-06-16 04:01:55,None,AUS
4,00049f68-a2a2-4d3d-bef6-f15c8c33a9dc,7885bef2-0959-4e88-878c-5eddb5850e1e,hospital,metropolitan centre,Intensive Care,adults,clinical practice,generally complex,moderate,always,2016-06-01,2017-02-27 20:46:37,2017-05-31,AUS
5,0008e1bc-589a-4bf3-ae16-ace555566578,ab2a1aed-322f-48b7-915d-7749ee02b65c,private practice,metropolitan centre,"Defence Nursing, GP nursing, small procedures,...",adults,clinical practice,very complex,high,always,2017-06-01,2017-06-04 12:46:32,2018-05-31,AUS
6,0009a8e2-e28f-47be-83fd-1f6d244d88c9,09f1c653-1127-43fa-a213-6b3515a8b21e,hospital,large rural centre,"Emergency,Critical Care",adults,clinical practice,very complex,moderate,always,2021-09-15,2021-05-14 04:21:07,None,AUS
7,000a1700-b302-415b-8c57-9ddcc1b61599,38bccb57-b74a-4cbf-9953-3879596b5e97,hospital,capital city,Clinical Nurse Specialist,Safety & Quality,management,be generally complex,moderate,always,2016-06-01,2016-10-17 23:16:22,2017-05-31,AUS
8,000a1b0d-5b7e-4ae1-a5a0-6fccab0c7559,e1c13c21-c769-4993-83b2-058373ce57d7,hospital,large rural centre,Orthopaedics,adults,clinical practice,generally complex,moderate,usually,2021-06-19,2021-06-18 22:28:44,None,AUS
9,000af191-1389-4cba-900e-e4a87debc02d,0da7e697-cef6-4493-a348-1160321ce2fe,hospital,capital city,Anaesthetic Nurse,children,clinical practice,generally complex,moderate,always,2018-04-23,2021-06-03 08:15:01,None,AUS
